<a href="https://colab.research.google.com/github/mzouros/dl_gmsa/blob/main/Greek_Music_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Run Colab localy

jupyter notebook \
--NotebookApp.allow_origin='https://colab.research.google.com' \
--port=8888 \
--NotebookApp.port_retries=0

In [ ]:
# Imports

import os
import glob
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time
import pandas as pd

import errno

In [ ]:
# Dataset size

mp3Path = '/home/mike/Artificial Intelligence MSc/2nd Semester/4. Deep Learning/MP3'
mp3Count = 0
wmaCount = 0
flacCount = 0
datasetAlbumNames = [] 
datasetTrackNames = []

for folder in os.listdir(mp3Path):
  trackCount = 0
  datasetAlbumNames.append(folder)
  songPath = mp3Path + '/' + folder
  for song in os.listdir(songPath):
    if song.lower().endswith(".mp3".lower()):
      # print(os.path.join(songPath, song))
      mp3Count += 1
    # for .wma  
    else:
      # print(os.path.join(songPath, song))
      wmaCount += 1
    datasetTrackNames.append(os.path.splitext(song)[0])
    trackCount += 1
  print(folder, trackCount)
print("")
print("Total songs found:", mp3Count+wmaCount+flacCount)

In [ ]:
# Spotify Developer Credentials

client_id = 'f1947444cbe9421e815b74a771326877'
client_secret = 'a1099b60916440aabc9a24f1583bb73e'

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [ ]:
# Get the IDs for each playlist of the user

userId = 'mdkyyoecyz3a2cxye8qjvt91l'
playlistIds = []
playlistNames = [] 
offset = 0

userPlaylists1 = sp.user_playlists(userId, offset=0) # 0 - 49
userPlaylists2 = sp.user_playlists(userId, offset=50) # 50 - 99
userPlaylists3 = sp.user_playlists(userId, offset=100) # 100 - 102
for item in userPlaylists1['items']:
  playlistIds.append(item['id'])
  playlistNames.append(item['name'])
for item in userPlaylists2['items']:
  playlistIds.append(item['id'])
  playlistNames.append(item['name'])
for item in userPlaylists3['items']:
  playlistIds.append(item['id'])
  playlistNames.append(item['name'])
print("Playlists' ids:", playlistIds)
print("Playlists' names:", playlistNames)
print("No of Playlists:", len(playlistNames))

In [ ]:
# Validate Spotify's playlists' names against dataset's albums' names
# If it prints nothing, we are good to go

datasetAlbumNames.sort()
playlistNames.sort()

for i in range(102):
  if playlistNames[i] != datasetAlbumNames[i]:
    print("Names [", playlistNames[i], "] and [", datasetAlbumNames[i], "] are not identical. Found on index:", i)

In [ ]:
# Get the IDs for each track of each playlist of the user

user = 'mzouros'
spotifyTrackIds = []
spotifyTrackNames = []

for id in playlistIds:
  # Check size of the playlist
  if sp.user_playlist_tracks(user, id)['total'] < 100:  
    playlist = sp.user_playlist_tracks(user, id)
    for item in playlist['items']:
      track = item['track']
      print(track['artists'][0]['name'], "---", track['name'], "-", track['id'])
      spotifyTrackIds.append(track['id'])
      spotifyTrackNames.append(track['name'])
  elif sp.user_playlist_tracks(user, id)['total'] < 200:
    offset = 0
    for x in range(2):
      playlist = sp.user_playlist_tracks(user, id, offset = offset)
      for item in playlist['items']:
        track = item['track']
        print(track['artists'][0]['name'], "---", track['name'], "-", track['id'])
        spotifyTrackIds.append(track['id'])
        spotifyTrackNames.append(track['name'])
      offset += 100
  else:
    offset = 0
    for x in range(3):
      playlist = sp.user_playlist_tracks(user, id, offset = offset)
      for item in playlist['items']:
        track = item['track']
        print(track['artists'][0]['name'], "---", track['name'], "-", track['id'])
        spotifyTrackIds.append(track['id'])
        spotifyTrackNames.append(track['name'])
      offset += 100

print(len(spotifyTrackIds))

In [ ]:
# Validate Spotify's playlists' tracks' names against dataset's albums' tracks' names
# If it prints zero, we are good to go

missedCount = 0
for track in datasetTrackNames: 
  track = track.replace("_", "/") # song titles with "slashes" on Spotify are saved with "underscore" in our dataset
  if track not in spotifyTrackNames:
    missedCount += 1
    print(track)
print(missedCount)

In [ ]:
# Get Tracks Information and dump them to .csv

tracks = []
for id in spotifyTrackIds:
  try:
    meta = sp.track(id)
    features = sp.audio_features(id)

    # meta
    name = meta['name']
    album = meta['album']['name']
    artist = meta['album']['artists'][0]['name']
    release_date = meta['album']['release_date']
    length = meta['duration_ms']
    popularity = meta['popularity']

    # features
    acousticness = features[0]['acousticness']
    danceability = features[0]['danceability']
    energy = features[0]['energy']
    instrumentalness = features[0]['instrumentalness']
    liveness = features[0]['liveness']
    loudness = features[0]['loudness']
    speechiness = features[0]['speechiness']
    tempo = features[0]['tempo']
    time_signature = features[0]['time_signature']

    track = [name, album, artist, release_date, length, popularity, danceability, acousticness, energy, instrumentalness, liveness, loudness, speechiness, tempo, time_signature]
    print(track)
    tracks.append(track)
  # We ensure that a faulty id or an empty JSON, won't crash the run  
  except ValueError:
    try:
      print("JSONDecodeError")
    except NameError:
      print("NameError")

# Create metadata/features .csv
df = pd.DataFrame(tracks, columns = ['name', 'album', 'artist', 'release_date', 'length', 'popularity', 'danceability', 'acousticness', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature'])
df.to_csv("Spotify_Tracks.csv", sep = ',')